In [1]:
import numpy as np
from random import random, seed
from copy import deepcopy
from queue import PriorityQueue
#from MyHeap import MyHeap

# Funkcje grafowe

In [2]:
def print_matrix(vertices, matrix):
    '''
    wypisuje na ekranie graf podany jako macierz sąsiedztwa
    '''
    n = len(matrix)
    if (vertices is not None) and (len(vertices) == n):
        vv = vertices
    else:
        vv = range(1, n+1)
    for i in range(n):
        print(vv[i], ":", end = "")
        for j in range(n):
            if matrix[i, j]:
                print(" ", vv[j], end="")
        print("")

In [3]:
def print_graph(graph):
    '''
    wypisuje na ekranie graf podany jako słownik w pythonie
    '''
    for v in graph:
        print(v, ":", end = "")
        for u in graph[v]:
            print("", u, end = "")
        print("")    

Tworzenie i modyfikacja grafów

In [4]:
def add_vertex(graph, vertex):
    '''
    Nowy wierzchołek do istniejącego grafu
    '''
    if vertex not in graph:
        graph[vertex] = []
        
def add_arc(graph, arc):
    '''
    Dodaje nowy łuk (podany jako para wierzchołków) do istniejącego grafu
    Rozważamy grafy proste, skierowane
    '''
    u, v = arc
    add_vertex(graph, u)
    add_vertex(graph, v)
    if v not in graph[u]:
        graph[u].append(v)

def add_edge(graph, edge):
    '''
    Dodaje nową krawędź (podaną jako parę wierzchołków) do istniejącego grafu
    traktując graf nieskierowany jako prosty graf skierowany, symetryczny i bez pętli.
    '''
    u, v = edge
    add_vertex(graph, u)
    add_vertex(graph, v)
    if u == v:
        raise ValueError("Pętla!")
    if v not in graph[u]:
        graph[u].append(v)
    if u not in graph[v]:
        graph[v].append(u)
    

Losowe grafy

In [5]:
def random_graph(n, p):
    '''
    losowanie grafu o n wierzchołkach z p-ństwem połączenia p
    '''
    random_graph = {}
    for i in range(1, n+1):
        add_vertex(random_graph, i)
        for j in range(1, i):
            if random() < p:
                add_edge(random_graph, [i, j])
    return random_graph

Konwersja między macierzą sąsiedztwa, a słownikiem

In [6]:
def matrix_to_dict(vertices, matrix):
    '''
    konwersja między postacią macierzową a słownikową
    '''
    n = len(vertices)
    graph = {}
    for vertex in vertices:
        graph[vertex] = []
    for i in range(n):    
        for j in range(n):
            if matrix[i, j] == 1:
                graph[list(graph.keys())[i]].append(list(graph.keys())[j])
    return graph            

In [7]:
def dict_to_matrix(graph, weighted = None):
    '''
    konwersja między postacią słownikową a macierzową
    '''
    n = len(graph)
    matrix = np.zeros((n,n))
    vertices = list(graph.keys())
    if weighted == None:
        for key in graph:
            for arc in graph[key]:
                i = list(graph.keys()).index(key)
                j = list(graph.keys()).index(arc)
                matrix[i, j] = 1
    else:
        for key in graph:
            for arc in graph[key]:
                i = list(graph.keys()).index(key)
                j = list(graph.keys()).index(arc[0])
                matrix[i, j] = arc[1]
    return (vertices, matrix)

Tworzenie cyklu o n wierzchołkach

In [8]:
def cycle(n):
    '''
    tworzenie cyklu o n wierzchołkach
    '''
    cycle = {}
    for i in range(1, n+1):
        add_vertex(cycle, i)
        cycle[i].append(i%n + 1)
    return cycle    

# Wczytywanie grafów z plików

In [9]:
def graph_from_edges(filename, directed = 0):
    '''
    wczytuje graf z pliku tekstowego (pełna ścieżka dostępu), który w każdej linii zawiera opis jednej krawędzi (pary słów),
    ewentualnie jednego wierzchołka (pojedyncze słowo). Jako wynik zwraca graf w formie listy sąsiedztwa
    '''
    graph = {}
    file = open(filename, "r")             #otwieranie pliku do odczytu
    for line in file:                      #dla każdej linii w pliku
        words = line.strip().split()       #rozbijam linię na słowa
        if len(words) == 1:                #jedno słowo = wierzchołek
            add_vertex(graph, words[0])
        elif len(words) == 2:              #więcej słów - używamy dwóch pierwszych
            if directed:
                add_arc(graph, (words[0],words[1]))
            else:
                add_edge(graph, (words[0],words[1]))
        elif len(words) > 2: # more than two words, weighted graph  
            if directed:
                add_arc(graph, (words[0],words[1]))
                graph[words[0]][-1] = (words[1], words[2])
            else:
                add_edge(graph, (words[0],words[1]))
                graph[words[0]][-1] = (words[1], words[2])
                graph[words[1]][-1] = (words[0], words[2])
    file.close()
    return graph


def graph_to_neighbourlist(graph, filename):
    '''
    zapisuje graf do pliku tekstowego (pełna ścieżka dostępu) jako listę sąsiedztwa 
    '''
    file = open(filename, 'w')   #otwarcie pliku do zapisu
    for v in graph:
        neigh_list = f"{v}:"
        for u in graph[v]:  
            neigh_list = neigh_list + f" {u}"  #u na koniec listy sąsiedztwa
        neigh_list = neigh_list + "\n"  #koniec wiersza
        file.write(neigh_list)
    file.close()
    
    
def graph_to_edges(graph, filename):
    '''
    zapisuje graf do pliku tekstowego (pełna ścieżka dostępu), który w każdej linii zawiera opis jednej krawędzi (pary słów),
    ewentualnie jednego wierzchołka (pojedyncze słowo). 
    '''
    file = open(filename, 'w')
    for v in graph:
        if len(graph[v]) == 0:
            edges = f"{v}" + "\n"
            file.write(edges)
        else:    
            for u in graph[v]:
                edges = f"{v} " + f"{u}" + "\n"
                file.write(edges)
    file.close()  
    
    
def graph_from_neighbourlist(filename):
    '''
    wczytuje graf z pliku tekstowego (pełna ścieżka dostępu), który ma postać listy sąsiedztwa. 
    Jako wynik zwraca graf w formie listy sąsiedztwa.
    '''
    graph = {}
    file = open(filename, 'r')
    for line in file:
        words = line.strip().split(':')
        words1 = words[1].split()
        add_vertex(graph, words[0])
        for v in words1:
            graph[words[0]].append(v)
    return graph

In [10]:
def Prufer(graph):
    '''
    Kod Prufera drzewa - zwrócony jako napis.
    Wymagamy, aby graf był drzewem.
    '''
    tr = deepcopy(graph)
    code = ""
    for i in range(len(graph) - 2):
        for x in sorted(tr): #po kolei przeglądam nieusunięte wierzchołki 
            if len(tr[x]) == 1: #najmniejszy liść
                break
        v = tr[x][0] #sąsiad najmniejszego x 
        code = code + f'{v} '
        tr[v].remove(x) #usuwam x z listy sąsiadów v
        tr.pop(x)       #usuwam x z drzewa
    return code.strip()

In [11]:
def tree_from_Prufer(code: str):
    '''
    tworzy drzewo z kodu Prufera
    '''
    tree = {}
    clist = [int(x) for x in code.strip().split()] #kod zamieniony na listę liczb
    n = len(clist) + 2  #liczba wierzchołków
    vert = [v for v in range(1, n+1)]  #lista liczb od 1 do n
    for v in vert:
        add_vertex(tree, v)
    for i in range(n-2):
        for x in vert:
            if not x in clist:   #najmniejszy liść
                break 
        v = clist.pop(0)    #usuwam pierwszy element (sąsiad x)
        add_edge(tree, (x, v))
        vert.remove(x)
    add_edge(tree, vert)
    return tree

In [12]:
def ConnectedComponents(graph):
    '''
    znajduje spójne składowe w grafie nieskierowanym, zwraca listę list wierzchołków
    uwaga: jako pierwszy element listy uzyskamy zbiór wszystkich wierzchołków grafu 
    '''
    def DFS(u):
        '''
        przeszukiwanie w głąb
        '''
        for w in graph[u]:
            if not w in VT[0]: #w jeszcze nieodwiedzony                          
                VT[0].add(w)   #już odwiedzony
                VT[-1].add(w)  #w ostatniej spójnej składowej
                DFS(w)
    '''
    VT - lista zbiorów VT[i] :
    dla i > 0 lista wierzchołków spójnych składowych 
    dla i = 0 - lista wszystkich odwiedzonych wierzchołków 
    '''
    VT = [set([])]
    for v in graph:
        if v not in VT[0]:
            VT[0].add(v)
            VT.append(set([v])) #zaczątek nowej spójnej składowej
            DFS(v)
    return VT        

In [13]:
def ConnectedComponentsGraphs(graph):
    '''
    zwraca listę grafów - spójnych składowych grafu podanego jako parametr
    '''
    components_list = ConnectedComponents(graph)
    graphs = []
    n = len(components_list)
    for i in range(1, n):
        copy_graph = deepcopy(graph)
        for v in components_list[0]:
            if v not in components_list[i]:
                copy_graph.pop(v)
        graphs.append(copy_graph)
    return graphs

In [14]:
def random_bipartite_graph(n, p):
    '''
    generuje dwudzielny graf losowy o 2n wierzchołkach
    '''
    random_bipartite_graph = {}
    for i in range(1, 2*n+1):
        add_vertex(random_bipartite_graph, i)
    for i in range(1, n+1):    
        for j in range(n+1, 2*n+1):
            if random() < p:
                add_edge(random_bipartite_graph, [i, j])
    return random_bipartite_graph

In [15]:
def preorder(graph, v):
    '''
    wypisanie drzewa w porządku preorder
    '''
    def DFS(u):
        for w in graph[u]:
            if not w in VT[0]:                           
                VT[0].add(w)   
                VT[1].append(w)  
                DFS(w)
    VT = [set([v]), [v]]
    DFS(v)
    return VT[1] 

In [16]:
def postorder(graph, v):
    '''
    wypisanie drzewa w porządku postorder
    '''
    def DFS(u):
        for w in graph[u]:
            if not w in VT[0]:                           
                VT[0].add(w)     
                DFS(w)
                VT[1].append(w)
    VT = [set([v]), []]
    DFS(v)
    VT[1].append(v)
    return VT[1] 

In [17]:
def Distance(graph, v):
    '''
    znajduje i zwraca jako wektor słownik odległości od wierzchołka v do wierzchołków w tej samej spójnej składowej co v
    '''
    dist = {v:0} #zalążek 
    kolejka = [v]
    while len(kolejka) > 0:
        u = kolejka.pop(0)
        for w in graph[u]:
            if not w in dist:
                dist[w] = dist[u] + 1
                kolejka.append(w)
    return dist   

In [18]:
def ConnectedComponentsBFS(graph):
    '''
    znajduje spójne składowe w grafie nieskierowanym, zwraca listę list wierzchołków
    uwaga: jako pierwszy element listy uzyskamy zbiór wszystkich wierzchołków grafu
    '''
    def BFS(u):
        '''
        przeszukiwanie grafu w szerz
        '''
        kolejka = [u]
        while len(kolejka) > 0:
            w = kolejka.pop(0)
            for k in graph[w]:
                if not k in VT[0]:
                    VT[0].add(k)   #już odwiedzony
                    VT[-1].add(k)  #w ostatniej spójnej składowej
                    kolejka.append(k)       
    VT = [set([])]
    for v in graph:
        if v not in VT[0]:
            VT[0].add(v)
            VT.append(set([v])) #zaczątek nowej spójnej składowej
            BFS(v)
    return VT   

In [19]:
def Floyd_Warshall(W):
    n = len(W)
    D = deepcopy(W)
    for k in range(n):
        for i in range(n):
            for j in range(n):
                D[i,j] = min(D[i,j], D[i,k]+D[k,j])
    return D

In [20]:
def GDFS(graph, order = None):
    '''
    uogólnione przechodzenie grafu skierowanego w głąb
    order - porządek wierzchołków (opcjonalny)
    zwraca dwa słowniki (visited, processed)
    '''
    def DFS(u):
        '''
        wewnętrzna - przeszukiwanie grafu w głąb
        '''
        visited[u] = len(visited) + 1 #u już odwiedzony
        for w in graph[u]:
            if not w in visited: #w jeszcze nieodwiedzony                          
                DFS(w)
        processed[u] = len(visited)    
    if order is None:
        vertices = graph.keys()
    else:
        vertices = order
    visited = {}
    processed = {}
    for v in vertices:
        if not v in visited:
            DFS(v)
    return visited, processed

In [21]:
def TopologicalSort(graph, order = None):
    '''
    sortowanie topologiczne grafu skierowanego
    '''
    lista = sorted(graph.keys())
    visited, processed = GDFS(graph, order) #można dać inny porządek np. lista
    n = len(graph)
    order = {}
    for v in graph:
        order[v] = n*processed[v] - visited[v]
    sort = sorted(order, key = order.get, reverse = True)
    return sort

In [22]:
def MinSpanningTree(graph):
    '''
    algorytm jarnika-prima -- minimalne drzewa spinające. Dla nieskierowanych grafów ważonych zwraca parę,
    gdzie waga to łączna waga drzewa, a drzewo to minimalne drzewo spinające w formie grafu ważonego.
    '''
    for v in graph:
        break
    tree = {v:[]} #zalążek drzewa
    weight = 0 #łączna waga
    q = PriorityQueue() #pusta kolejka priorytetowa
    for (u, w) in graph[v]:
        q.put((int(w), v, u))
    while not q.empty():
        (w, v, u) = q.get()
        if u not in tree:
            weight += w
            tree[u] = [(v, w)]
            tree[v].append([(u, w)])
            for (x, w) in graph[u]:
                if not x in tree:
                    q.put((int(w), u, x))
    if len(tree) < len(graph):
        print('Graf niespójny - zwrócone drzewo dla jednej składowej')
    return weight, tree

In [23]:
def direct_dist(graph, weighted = True, infinity = True):
    n = len(graph)
    if infinity == True:
        inf = float('inf')
        D = np.array([[inf for i in range(n)] for j in range(n)])
    else:
        D = np.array([[0 for i in range(n)] for j in range(n)])
    if weighted == False:
        for key in graph:
            for arc in graph[key]:
                i = list(graph.keys()).index(key)
                j = list(graph.keys()).index(arc)
                D[i, i] = 0
                D[i, j] = 1
    else:
        for key in graph:
            for arc in graph[key]:
                i = list(graph.keys()).index(key)
                j = list(graph.keys()).index(arc[0])
                D[i, i] = 0
                D[i, j] = arc[1]
    return D

In [24]:
def Dijkstra(graph, s):
    """
    Algorytm Dijkstry - najkrótsze ścieżki z jednym źródłem (z wierzchołka s)
    Wagi krawędzi - liczby naturalne
    """
    dist = {}
    pred = {}
    for v in graph:
        dist[v] = 2**31 # substytut nieskończoności
        pred[v] = None
    dist[s] = 0
    q = PriorityQueue()
    q.put((0, s))
    V = set() # zbiór wierzchołków przetworzonych
    while not q.empty():   # dopóki kolejka niepusta
        (d, u) = q.get()      # odległość i najbliższy wierzchołek
        if u not in V:
            V.add(u)
            for (v, w) in graph[u]:  # relax
                if dist[v] > dist[u] + int(w):
                    dist[v] = dist[u] + int(w)
                    pred[v] = u
                    q.put((dist[v], v))
    return dist, pred

In [25]:
def Dijkstra2(graph, s):
        """Algorytm Dijkstry - szukanie najkrótszych scieżek z jednym źródłem
            wariant z własnym kopcem - każdy wierzchołek wstawiony raz"""
        heap = MyHeap()
        # init
        dist = {}
        pred = {}
        for v in graph:
            dist[v] = 2**31 # taki substytut nieskończoności
            pred[v] = None
        dist[s] = 0
        # główny algorytm
        heap.insert_val(s, 0)
        V = set()
        while len(heap):    # dopóki kopiec niepusty, są w nim tylko wierzchołki osiągalne z s
            (d, u) = heap.get_min()
            V.add(u)
            for (v, w) in graph[u]:
                if v not in V: # relax
                    if dist[v] > dist[u] + int(w):
                        dist[v] = dist[u] + int(w)
                        pred[v] = u
                        heap.insert_val(v, dist[v]) #insert_val działa tak, że jak v jest to zmniejsza wartość, a jak nie ma to dodaje
        return dist, pred

In [26]:
def Bellman_Ford(graph, s):
    # init
    dist = {}
    pred = {}
    for v in graph:
        dist[v] = 2**31 # taki substytut nieskończoności
        pred[v] = None
    dist[s] = 0
    for i in range(len(graph)-1):
        for u in graph:
            for (v, w) in graph[u]:  # relax
                    if dist[v] > dist[u] + int(w):
                        dist[v] = dist[u] + int(w)
                        pred[v] = u
    for u in graph:
        for (v, w) in graph[u]:  
                if dist[v] > dist[u] + int(w):
                    print(f"Graf zawiera cykl o ujemnej wadze {u} -> {v}!") 
                    return None, None
    return dist, pred

In [35]:
def flow_EK(C, s, t):
        n = len(C) # C is the capacity matrix
        F = np.array([[0] * n for i in range(n)]) #F is the flow matrix 
        path = BFS(C, F, s, t)
        while path != None:
            flow = min(C[u, v] - F[u, v] for u,v in path)
            for u,v in path:
                F[u, v] += flow
                F[v, u] -= flow
            path = BFS(C, F, s, t)
        return sum(F[s, i] for i in range(n))

#find path by using BFS
def BFS(C, F, s, t):
        n = len(C)
        q = PriorityQueue()
        q.put((0, s))
        paths = {s:[]}
        if s == t:
            return paths[s] 
        while not q.empty():
            (d, u) = q.get()
            for v in range(n):
                    if (C[u, v] - F[u, v] > 0) and v not in paths:
                        paths[v] = paths[u] + [(u,v)]
                        if v == t:
                            return paths[v]
                        q.put((0, v))
        return None

# Przykłady wykorzystania

In [37]:
C = np.array([[ 0, 3, 3, 0, 0, 0 ],  # s
              [ 0, 0, 2, 3, 0, 0 ],  # o
              [ 0, 0, 0, 0, 2, 0 ],  # p
              [ 0, 0, 0, 0, 4, 2 ],  # q
              [ 0, 0, 0, 0, 0, 2 ],  # r
              [ 0, 0, 0, 0, 0, 3 ]]) # t
              # s, o, p, q, r, t
    
source = 0  # s
sink = 5    # t
max_flow_value = flow_EK(C, source, sink)
print("Edmonds-Karp algorithm")
print("max_flow_value is: ", max_flow_value)

Edmonds-Karp algorithm
max_flow_value is:  4


# Sortowanie topologiczne

In [158]:
ubranie_graph = graph_from_edges('ubranie.txt', directed = 1)
porz = TopologicalSort(ubranie_graph)
print(porz)

['koszula', 'krawat', 'skarpety', 'slipki', 'kalesony', 'spodnie', 'szelki', 'marynarka', 'plaszcz', 'buty']


# Small world phenomenon

In [37]:
import sys

n = 500
p = 1/100
#sys.setrecursionlimit(n+5)

rgraph = random_graph(n,p)
lista = ConnectedComponentsGraphs(rgraph)
graph = lista[0]
print(len(graph))

496


In [38]:
md = {}
ecc = {}
for v in graph:
    dist = Distance(graph, v)
    ecc[v] = max(dist.values())
    md[v] = sum(dist.values())/len(dist.values())
print('Promień:', min(ecc.values()), '; Średnica:', max(ecc.values()), '; Średnia:', sum(md.values())/len(md.values()))

Promień: 5 ; Średnica: 8 ; Średnia: 4.003755853277837


## To samo przy użyciu algorytmu Floyda-Warshalla (ale wolniej) 

In [39]:
W = direct_dist(graph)

In [40]:
W

array([[ 0.,  1., inf, ..., inf, inf, inf],
       [ 1.,  0., inf, ..., inf, inf, inf],
       [inf, inf,  0., ..., inf, inf, inf],
       ...,
       [inf, inf, inf, ...,  0., inf, inf],
       [inf, inf, inf, ..., inf,  0., inf],
       [inf, inf, inf, ..., inf, inf,  0.]])

In [41]:
F = Floyd_Warshall(W) #nie jest szybsze

In [42]:
F

array([[0., 1., 4., ..., 5., 4., 4.],
       [1., 0., 5., ..., 4., 5., 5.],
       [4., 5., 0., ..., 2., 5., 4.],
       ...,
       [5., 4., 2., ..., 0., 4., 4.],
       [4., 5., 5., ..., 4., 0., 4.],
       [4., 5., 4., ..., 4., 4., 0.]])

In [43]:
md1 = {}
ecc1 = {}
for i in range(len(F)):
    ecc1[i] = max(F[i])
    md1[i] = sum(F[i])/len(F[i])
print('Promień:', min(ecc1.values()), '; Średnica:', max(ecc1.values()), '; Średnia:', sum(md1.values())/len(md1.values()))

Promień: 5.0 ; Średnica: 8.0 ; Średnia: 4.003755853277837


# Dalsze przykłady

In [232]:
prufer_tree = tree_from_Prufer('1 2 7 3 4')

In [233]:
print_graph(prufer_tree)

1 : 5 2
2 : 1 7
3 : 6 4
4 : 3 7
5 : 1
6 : 3
7 : 2 4


In [234]:
preorder(prufer_tree, 1)

[1, 5, 2, 7, 4, 3, 6]

In [235]:
postorder(prufer_tree, 1)

[5, 6, 3, 4, 7, 2, 1]

In [148]:
random_graph = random_bipartite_graph(5, 1/5)

In [149]:
print_graph(random_graph)

1 : 6 9
2 : 8
3 : 7 9 10
4 : 6 9
5 : 10
6 : 1 4
7 : 3
8 : 2
9 : 1 3 4
10 : 3 5


In [24]:
graph = random_graph(10, 1/5)
print_graph(graph)
print('----------------------------------------------------------')

1 : 3
2 :
3 : 1 7 8
4 : 5 6 9
5 : 4 10
6 : 4
7 : 3 9 10
8 : 3
9 : 4 7 10
10 : 5 7 9
----------------------------------------------------------


In [25]:
print(ConnectedComponentsBFS(graph))

[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}, {1, 3, 4, 5, 6, 7, 8, 9, 10}, {2}]


In [111]:
for i in range(len(ConnectedComponentsGraphs(graph))):
    print_graph(ConnectedComponentsGraphs(graph)[i])
    print('-----------------------------------------')

1 : 9
2 : 9 10
4 : 5 7 10
5 : 4 10
7 : 4
8 : 10
9 : 1 2
10 : 2 4 5 8
-----------------------------------------
3 :
-----------------------------------------
6 :
-----------------------------------------


In [32]:
vertices = ["a", "b", "c", "d", "e", "i"]
matrix = np.array([[0,1,1,0,0,0],[1,0,1,0,0,0],[1,1,0,0,1,0],[0,0,1,0,1,1],[0,0,1,1,0,1],[1,0,1,0,1,0]])

In [9]:
print(vertices)
print(matrix)
print("------------------------------------------------------")
print_matrix(vertices, matrix)
print("------------------------------------------------------")
print_matrix(None, matrix)

['a', 'b', 'c', 'd', 'e', 'i']
[[0 1 1 0 0 0]
 [1 0 1 0 0 0]
 [1 1 0 0 1 0]
 [0 0 1 0 1 1]
 [0 0 1 1 0 1]
 [1 0 1 0 1 0]]
------------------------------------------------------
a :  b  c
b :  a  c
c :  a  b  e
d :  c  e  i
e :  c  d  i
i :  a  c  e
------------------------------------------------------
1 :  2  3
2 :  1  3
3 :  1  2  5
4 :  3  5  6
5 :  3  4  6
6 :  1  3  5


Druga reprezentacja

In [65]:
graph = {
    'a' :  ['b',  'c'],
    'b' :  ['a',  'c'],
    'c' :  ['a',  'b',  'e'],
    'd' :  ['c',  'e',  'i'],
    'e' :  ['c',  'd',  'i'],
    'i' :  ['a',  'c',  'e'],
    'g' :  []
}
print(graph)

{'a': ['b', 'c'], 'b': ['a', 'c'], 'c': ['a', 'b', 'e'], 'd': ['c', 'e', 'i'], 'e': ['c', 'd', 'i'], 'i': ['a', 'c', 'e'], 'g': []}


In [11]:
print_graph(graph)

a : b c
b : a c
c : a b e
d : c e i
e : c d i
i : a c e
g :


In [12]:
add_vertex(graph, "h")
print_graph(graph)

a : b c
b : a c
c : a b e
d : c e i
e : c d i
i : a c e
g :
h :


In [13]:
add_arc(graph, ["u", "v"])
print_graph(graph)

a : b c
b : a c
c : a b e
d : c e i
e : c d i
i : a c e
g :
h :
u : v
v :


In [14]:
add_edge(graph, ["h", "f"])
add_edge(graph, ["h", "i"])
print_graph(graph)

a : b c
b : a c
c : a b e
d : c e i
e : c d i
i : a c e h
g :
h : f i
u : v
v :
f : h


Tworzenie grafów losowych G(n,p)

In [15]:
seed(2024)

In [16]:
#losuje graf G(10,1/3)
random_g = random_graph(10,1/3) 
print_graph(random_g)

1 : 5 10
2 : 3 5 9 10
3 : 2 8
4 : 6 7 8 10
5 : 1 2 10
6 : 4 8 10
7 : 4 9 10
8 : 3 4 6
9 : 2 7
10 : 1 2 4 5 6 7


In [46]:
graph = np.array([[0,1,1,0,0,0], [1,0,1,0,0,0], [1,1,0,0,1,0], [0,0,1,0,1,1], [0,0,1,1,0,1], [1,0,1,0,1,0]])

In [18]:
print_graph(matrix_to_dict(graph))

1 : 2 3
2 : 1 3
3 : 1 2 5
4 : 3 5 6
5 : 3 4 6
6 : 1 3 5


In [24]:
print_graph(cycle(6))

1 : 2
2 : 3
3 : 4
4 : 5
5 : 6
6 : 1


In [84]:
vertices, matrix = dict_to_matrix(graph)
print_graph(matrix_to_dict(vertices, matrix))

a : b c
b : a c
c : a b e
d : c e i
e : c d i
i : a c e
g :


In [10]:
%%writefile lista.txt
A   B
B C
B D
D C
E 
F

Overwriting lista.txt


In [63]:
graph1 = graph_from_edges("lista.txt")
print_graph(graph1)

A : B
B : A C D
C : B D
D : B C
E :
F :


In [81]:
add_edge(graph1, ("A", "E"))
add_edge(graph1, ("E", "F"))
add_vertex(graph1, 'G')

In [82]:
print_graph(graph1)

A : B E
B : A C D
C : B D
D : B C
E : A F
F : E
G :


In [83]:
graph_to_neighbourlist(graph1, "graf1.txt")

In [84]:
print_graph(graph_from_neighbourlist('graf1.txt'))

A : B E
B : A C D
C : B D
D : B C
E : A F
F : E
G :


In [12]:
file = open('lista.txt', 'r')
for line in file:
    words = line.split()
    print(words)

['A', 'B']
['B', 'C']
['B', 'D']
['D', 'C']
['E']
['F']


In [16]:
tree = {
    '1' :  ['2'],
    '2' :  ['1',  '3',  '4'],
    '3' :  ['2'],
    '4' :  ['2'],
}

In [17]:
print(Prufer(tree))

2 2


In [165]:
print_graph(tree_from_Prufer('1 2 7 3 4'))

1 : 5 2
2 : 1 7
3 : 6 4
4 : 3 7
5 : 1
6 : 3
7 : 2 4


In [170]:
prufer_tree = tree_from_Prufer('1 2 7 3 4')

In [24]:
print(Prufer(tree_from_Prufer('1 2 7 3 4')))

1 2 7 3 4


In [25]:
print_graph(tree_from_Prufer('1 1 7 7 6 2'))

1 : 3 4 7
2 : 6 8
3 : 1
4 : 1
5 : 7
6 : 7 2
7 : 1 5 6
8 : 2


In [26]:
print(Prufer(tree_from_Prufer('1 1 7 7 6 2')))

1 1 7 7 6 2


In [59]:
import urllib.request
url = 'https://raw.githubusercontent.com/pgordin/OptDisc2024/main/wagi0.txt'
filename = 'wagi0.txt'
urllib.request.urlretrieve(url, filename)

('wagi0.txt', <http.client.HTTPMessage at 0x19daf8847c0>)

In [116]:
wgraph = graph_from_edges(filename, directed = 0)

In [123]:
print_graph(wgraph)

A : ('B', '3') ('E', '10')
B : ('A', '3') ('C', '26') ('D', '12')
E : ('A', '10') ('D', '7') ('F', '8') ('H', '4')
C : ('B', '26') ('D', '17') ('F', '13') ('G', '14')
D : ('B', '12') ('C', '17') ('E', '7') ('F', '15')
F : ('C', '13') ('D', '15') ('E', '8') ('G', '9') ('H', '6')
G : ('C', '14') ('F', '9') ('H', '16') ('I', '11')
H : ('E', '4') ('F', '6') ('G', '16')
I : ('G', '11')


In [89]:
weight, tree = MinSpanningTree(wgraph)
print(weight)
print_graph(tree)

63
A : [('B', 3)] [('E', 10)]
B : ('A', 3)
E : ('A', 10) [('H', 4)] [('D', 7)]
H : ('E', 4) [('F', 6)]
F : ('H', 6) [('G', 9)] [('C', 13)]
D : ('E', 7)
G : ('F', 9) [('I', 11)]
I : ('G', 11)
C : ('F', 13)


In [180]:
D = direct_dist(wgraph, weighted = 1)
D

array([[ 0.,  3., 10., inf, inf, inf, inf, inf, inf],
       [ 3.,  0., inf, 26., 12., inf, inf, inf, inf],
       [10., inf,  0., inf,  7.,  8., inf,  4., inf],
       [inf, 26., inf,  0., 17., 13., 14., inf, inf],
       [inf, 12.,  7., 17.,  0., 15., inf, inf, inf],
       [inf, inf,  8., 13., 15.,  0.,  9.,  6., inf],
       [inf, inf, inf, 14., inf,  9.,  0., 16., 11.],
       [inf, inf,  4., inf, inf,  6., 16.,  0., inf],
       [inf, inf, inf, inf, inf, inf, 11., inf,  0.]])

In [181]:
Floyd_Warshall(D)

array([[ 0.,  3., 10., 29., 15., 18., 27., 14., 38.],
       [ 3.,  0., 13., 26., 12., 21., 30., 17., 41.],
       [10., 13.,  0., 21.,  7.,  8., 17.,  4., 28.],
       [29., 26., 21.,  0., 17., 13., 14., 19., 25.],
       [15., 12.,  7., 17.,  0., 15., 24., 11., 35.],
       [18., 21.,  8., 13., 15.,  0.,  9.,  6., 20.],
       [27., 30., 17., 14., 24.,  9.,  0., 15., 11.],
       [14., 17.,  4., 19., 11.,  6., 15.,  0., 26.],
       [38., 41., 28., 25., 35., 20., 11., 26.,  0.]])

In [65]:
wgraph2 = graph_from_edges('wagi2.txt', directed = 1)

In [66]:
print_graph(wgraph2)

1 : ('2', '3') ('3', '8') ('5', '-4')
2 : ('4', '1') ('5', '7')
3 : ('2', '4')
5 : ('4', '6')
4 : ('1', '2') ('3', '-5')


In [67]:
Dijkstra(wgraph2, '1')

({'1': 0, '2': 1, '3': -3, '5': -4, '4': 2},
 {'1': None, '2': '3', '3': '4', '5': '1', '4': '5'})

In [68]:
Bellman_Ford(wgraph2, '1')

({'1': 0, '2': 1, '3': -3, '5': -4, '4': 2},
 {'1': None, '2': '3', '3': '4', '5': '1', '4': '5'})

# EK-flow test

In [31]:
wgraph3 = graph_from_edges('flow1.txt', directed = 1)

In [32]:
print_graph(wgraph3)

s : ('a', '6') ('b', '4')
a : ('b', '5') ('c', '2') ('d', '3')
b : ('c', '6')
c : ('a', '2') ('d', '3') ('t', '4')
d : ('t', '6')
t :


In [38]:
C = direct_dist(wgraph3, weighted = True, infinity = False)
C

array([[0, 6, 4, 0, 0, 0],
       [0, 0, 5, 2, 3, 0],
       [0, 0, 0, 6, 0, 0],
       [0, 2, 0, 0, 3, 4],
       [0, 0, 0, 0, 0, 6],
       [0, 0, 0, 0, 0, 0]])

In [39]:
source = 0  # s
sink = 5    # t
max_flow_value = flow_EK(C, source, sink)
print("Edmonds-Karp algorithm")
print("max_flow_value is: ", max_flow_value)

Edmonds-Karp algorithm
max_flow_value is:  10
